In [1]:
import scrapy

In [4]:
!pip install handler


  Running setup.py bdist_wheel for handler ... done
  Stored in directory: /Users/Lucia/Library/Caches/pip/wheels/17/3c/36/789c95c290a96f751a9ca50c654765dffebd5e37fc7188c72a
Successfully built handler


In [10]:
class HandlerItem(scrapy.Item):
        # define the fields for your item here like:
        # name = scrapy.Field()
        link = scrapy.Field() # urls from the start page
        link2 = scrapy.Field() # urls from secondary pages
        domainlink = scrapy.Field() # domain links from the first page, for scraping
        twitterlink = scrapy.Field() # twitter links from the first page
        twitterlink2 = scrapy.Field() # twitter links from the secondary pages
        twitterlinku = scrapy.Field() # cumulative list of twitter links
        handles = scrapy.Field() # twitter links truncated into handles
        pass

In [12]:
# -*- coding: utf-8 -*-
import handler
from handler.item import HandlerItem



ModuleNotFoundError: No module named 'handler.item'; 'handler' is not a package

In [ ]:
domain = 'twitter.com'
start = 'http://twitter.com'

class HouseSpider(scrapy.Spider):
    name = 'handler'
    allowed_domains = [domain]
    start_urls = [start]

    def parse(self, response):
        item = HandlerItem()
        item['link'] = []
        item['link2'] = []
        item['domainlink'] = []
        item['domainlink2'] = []
        item['twitterlink'] = []
        item['twitterlink2'] = []
        item['twitterlinku'] = []
        item['handles'] = []

        item['link'] = response.xpath('//a/@href').extract()

        item['twitterlink'] = [w for w in item['link'] if "twitter.com" in w]
        item['domainlink'] = [w for w in item['link'] if domain in w]
        for url in item['twitterlink']:
            item['twitterlinku'].append(url)
        for url in item['domainlink']:
            request = scrapy.Request(url, callback=self.parse2)
            request.meta['item'] = item
            yield request

        for link in item['twitterlinku']:
            handle = link.split(".com/")[1]
            if "/" in handle:
                handle = handle.split("/")[0]
            item['handles'].append(handle)
        yield item
        print (item['handles'])

    def parse2(self, response):
        item = response.meta['item']
        item['link2'] = response.xpath('//a/@href').extract()
        item['twitterlink2'] = [w for w in item['link2'] if "twitter.com" in w]
        for url in item['twitterlink2']:
            item['twitterlinku'].append(url)
            print (url)
        yield item['twitterlinku']
